<a id='report-scopes'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Report Scopes (IFRS 17 Methodology Business Logic)</span> </p>

This notebook contains the logic used to perform calculations upon reporting of data.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "ReportStorage"

# Scopes

## Helper Scopes

### Universe

In [0]:
public interface IUniverse: IScopeWithStorage<ReportStorage> {}

## Data Transformation

### Retrieve Raw Data

In [0]:
public interface Data: IScope<(ReportIdentity ReportIdentity, string EstimateType), ReportStorage>, IDataCube<ReportVariable> {
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<Data>(s => s.WithApplicability<DataWrittenActual>(x => x.GetStorage().EstimateTypesWithoutAoc.Contains(x.Identity.EstimateType)));
    
    protected IDataCube<ReportVariable> RawData => GetStorage().GetVariables(Identity.ReportIdentity, Identity.EstimateType);

    private IDataCube<ReportVariable> RawEops => RawData.Filter(("VariableType", AocTypes.EOP));
    private IDataCube<ReportVariable> NotEopsNotCls => RawData.Filter(("VariableType", "!EOP"),("VariableType", "!CL")); // TODO negation must be hardcoded (also to avoid string concatenation)
    
    private IDataCube<ReportVariable> CalculatedCl => (RawEops - NotEopsNotCls)
                                                        .AggregateOver(nameof(Novelty), nameof(VariableType))
                                                        .SelectToDataCube(x => Math.Abs(x.Value) >= Precision, x => x with { Novelty = Novelties.C, VariableType = AocTypes.CL });
    
    private IDataCube<ReportVariable> CalculatedEops => (NotEopsNotCls + CalculatedCl)
                                                        .AggregateOver(nameof(Novelty), nameof(VariableType))
                                                        .SelectToDataCube(x => x with { VariableType = AocTypes.EOP, Novelty = Novelties.C });
      
    IDataCube<ReportVariable> Data => NotEopsNotCls + CalculatedCl + CalculatedEops;
}
public interface DataWrittenActual: Data {
    IDataCube<ReportVariable> Data.Data => RawData;
}

### Foreign Exchange (FX) Rates

In [0]:
public interface Fx: IScope<(string ContractualCurrency, string FunctionalCurrency, FxPeriod FxPeriod, (int, int) Period, CurrencyType CurrencyType), ReportStorage> {   
    private double groupFxRate => Identity.CurrencyType switch {
            CurrencyType.Group => GetStorage().GetFx(Identity.Period, Identity.FunctionalCurrency, GroupCurrency, FxPeriod.Average),
            _ => 1
    };
    
    private double GetFunctionalFxRate(FxPeriod fxPeriod) => Identity.CurrencyType switch {
            CurrencyType.Contractual => 1,
            _ => GetStorage().GetFx(Identity.Period, Identity.ContractualCurrency, Identity.FunctionalCurrency, fxPeriod)
    };
    
    double Fx => GetFunctionalFxRate(Identity.FxPeriod) * groupFxRate;
}

In [0]:
public interface FxData: IScope<(ReportIdentity ReportIdentity, CurrencyType CurrencyType, string EstimateType), ReportStorage>, IDataCube<ReportVariable> {
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<FxData>(s => s.WithApplicability<FxDataWrittenActual>(x => x.GetStorage().EstimateTypesWithoutAoc.Contains(x.Identity.EstimateType)));
    
    protected IDataCube<ReportVariable> Data => GetScope<Data>((Identity.ReportIdentity, Identity.EstimateType)).Data
        .SelectToDataCube(x => Multiply( GetScope<Fx>((Identity.ReportIdentity.ContractualCurrency, 
                                                       Identity.ReportIdentity.FunctionalCurrency, 
                                                       GetStorage().GetFxPeriod(GetStorage().Args.Period, x.VariableType, x.Novelty),
                                                       (Identity.ReportIdentity.Year, Identity.ReportIdentity.Month),
                                                       Identity.CurrencyType)).Fx, x ) with { Currency = Identity.CurrencyType switch {
                                                                                                                    CurrencyType.Contractual => x.ContractualCurrency,
                                                                                                                    CurrencyType.Functional => x.FunctionalCurrency,
                                                                                                                    _ => GroupCurrency }});
    
    private IDataCube<ReportVariable> Eops => Data.Filter(("VariableType", AocTypes.EOP));
    private IDataCube<ReportVariable> NotEops => Data.Filter(("VariableType", "!EOP")); // TODO negation must be hardcoded (also to avoid string concatenation)
    
    private IDataCube<ReportVariable> Fx => (Eops - NotEops)
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(x => Math.Abs(x.Value) >= Precision, x => x with { Novelty = Novelties.C, VariableType = AocTypes.FX });
    
    IDataCube<ReportVariable> FxData => Data + Fx;
}

public interface FxDataWrittenActual: FxData {
    IDataCube<ReportVariable> FxData.FxData => Data;
}

## Present Value

<a id='best-estimate'></a>
### Best Estimate

Best Estimate of Present Value (PV) report includes contributions from the [Present Value](../Import/ImportScopeCalculation#present-values) calculation. The result of the Locked-in calculation is used when the Valuation Approach is BBA and the flag for Oci is active.

In [0]:
public interface BestEstimate: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> BestEstimate => Identity.Id switch {
            { ValuationApproach: ValuationApproaches.BBA, IsOci: true } => GetScope<LockedBestEstimate>(Identity).LockedBestEstimate,
            _ => GetScope<CurrentBestEstimate>(Identity).CurrentBestEstimate };
}

public interface LockedBestEstimate: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> LockedBestEstimate => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.BE)).FxData
        .Filter(("LiabilityType", Identity.Id.LiabilityType), ("EconomicBasis", EconomicBases.L));
}

public interface CurrentBestEstimate: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> CurrentBestEstimate => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.BE)).FxData
        .Filter(("LiabilityType", Identity.Id.LiabilityType), ("EconomicBasis", EconomicBases.C));
}

<a id='risk-adjustment'></a>
### Risk Adjustment

Risk Adjustment (RA) report includes contributions from the [Risk Adjustment](../Import/ImportScopeCalculation#risk-adjustment) calculation. The result of the Locked-in calculation is used when the Valuation Approach is BBA and the flag for Oci is active.

In [0]:
public interface RiskAdjustment: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> RiskAdjustment => Identity.Id switch {
            { ValuationApproach: ValuationApproaches.BBA, IsOci: true } => GetScope<LockedRiskAdjustment>(Identity).LockedRiskAdjustment,
            _ => GetScope<CurrentRiskAdjustment>(Identity).CurrentRiskAdjustment };
}

public interface LockedRiskAdjustment: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> LockedRiskAdjustment => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.RA)).FxData
        .Filter(("LiabilityType", Identity.Id.LiabilityType), ("EconomicBasis", EconomicBases.L));
}

public interface CurrentRiskAdjustment: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> CurrentRiskAdjustment =>  GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.RA)).FxData
        .Filter(("LiabilityType", Identity.Id.LiabilityType), ("EconomicBasis", EconomicBases.C));
}

<a id='fulfillment-cash-flows'></a>
## Fulfillment Cash flows
Fulfillment Cash flows (FCF) report includes contributions from the [Best Estimate](#best-estimate) of Present Value and [Risk Adjustment](#risk-adjustment) Present Value.

In [0]:
public interface Fcf: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> BestEstimate => GetScope<BestEstimate>(Identity).BestEstimate;
    private IDataCube<ReportVariable> RiskAdjustment => GetScope<RiskAdjustment>(Identity).RiskAdjustment;
    
    IDataCube<ReportVariable> Fcf => BestEstimate + RiskAdjustment;
}

public interface CurrentFcf: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {  
    private IDataCube<ReportVariable> BestEstimate => GetScope<CurrentBestEstimate>(Identity).CurrentBestEstimate;
    private IDataCube<ReportVariable> RiskAdjustment => GetScope<CurrentRiskAdjustment>(Identity).CurrentRiskAdjustment;
    
    IDataCube<ReportVariable> CurrentFcf => BestEstimate + RiskAdjustment;
}

public interface LockedFcf: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {   
    private IDataCube<ReportVariable> BestEstimate => GetScope<LockedBestEstimate>(Identity).LockedBestEstimate;
    private IDataCube<ReportVariable> RiskAdjustment => GetScope<LockedRiskAdjustment>(Identity).LockedRiskAdjustment;
    
    IDataCube<ReportVariable> LockedFcf => BestEstimate + RiskAdjustment;
}

<a id='technical-margin'></a>
## LRC Technical Margin: CSM, LC, LoReCo

The LRC [Technical Margin](../Import/ImportScopeCalculation#technical-margin) is allocated to either Contractual Service Margin (CSM) or Loss Component (LC) or Loss Recovery Component (LoReCo). Therefore, the correponding report is available only for Liability for Remaining Coverage (LRC) and includes contributions from the calculation of [CSM](../Import/ImportScopeCalculation#csm), [LC](../Import/ImportScopeCalculation#csm), and [LoReCo](../Import/ImportScopeCalculation#csm).

In [0]:
public interface Csm: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> Csm => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.C)).FxData;
}

In [0]:
public interface Lc: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> Lc => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.L)).FxData;
}

In [0]:
public interface Loreco: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> Loreco => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.LR)).FxData;
}

In [0]:
public interface LrcTechnicalMargin: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> Csm => GetScope<Csm>(Identity).Csm;
    private IDataCube<ReportVariable> Lc => GetScope<Lc>(Identity).Lc;
    private IDataCube<ReportVariable> Loreco => GetScope<Loreco>(Identity).Loreco;
    
    IDataCube<ReportVariable> LrcTechnicalMargin => Lc + Loreco - 1 * Csm;
}

<a id='written-accrual-deferral'></a>
## Written, Accruals and Deferrals 

Written Actual (Actual) report includes contributions from the [Actual](../Import/ImportScopeCalculation#actuals) import. 
<br> Accrual Actuals (Advance, Overdue Actual) report includes contributions from the [Advance](../Import/ImportScopeCalculation#advance-actuals) and [Overdue](../Import/ImportScopeCalculation#overdue-actuals) Actual calculation.
<br>Deferral (Deferral Actual) report includes contributions from the calculation of [Deferral](../Import/ImportScopeCalculation#deferrable-actuals) Actual.

In [0]:
public interface WrittenAndAccruals: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> Written => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.A)).FxData;
    IDataCube<ReportVariable> Advance => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.AA)).FxData;
    IDataCube<ReportVariable> Overdue => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.OA)).FxData;
}

In [0]:
public interface Deferrals: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> Deferrals => GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.DA)).FxData;
}

## Experience Adjustment

Experience Adjustment (EA) report includes contributions from the [Written](#written-accrual-deferral) Actual report and the [Best Estimate](#best-estimate) of Present Value report (only the *Cash flow* AoC Step). 

In [0]:
public interface ExperienceAdjustment: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> WrittenCashflow => GetScope<WrittenAndAccruals>(Identity).Written
        .Filter(("VariableType", AocTypes.CF));
    
    private IDataCube<ReportVariable> BestEstimateCashflow => GetScope<BestEstimate>(Identity).BestEstimate
        .Filter(("VariableType", AocTypes.CF))
        .SelectToDataCube(rv => rv with { EconomicBasis = null, Novelty = Novelties.C });

    IDataCube<ReportVariable> ActuarialExperienceAdjustment => WrittenCashflow - BestEstimateCashflow;
}

<a id='lic-actuarial'></a>
## LIC Actuarial (Actuarial Liability for Incurred Claims)

Actuarial Liability of Incurred Claims (LIC Actuarial) report includes the contributions from [Fulfillment cash flow](#fulfillment-cash-flows).

In [0]:
public interface LicActuarial: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    IDataCube<ReportVariable> LicActuarial => GetScope<CurrentFcf>(Identity).CurrentFcf.Filter(("LiabilityType", LiabilityTypes.LIC));
}

<a id='lic'></a>
## LIC (Liability for Incurred Claims)

Liability of Incurred Claims (LIC) report includes the contributions from [LIC Actuarial](#lic-actuarial) (Fullfilment Cash flow), [Advance](#written-accrual-deferral) Actual, and [Overdue](#written-accrual-deferral) Actual.

In [0]:
public interface Lic: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> licActuarial => GetScope<LicActuarial>(Identity).LicActuarial;
    private IDataCube<ReportVariable> accrual => GetScope<WrittenAndAccruals>(Identity).Advance.Filter(("LiabilityType", LiabilityTypes.LIC)) + 
        GetScope<WrittenAndAccruals>(Identity).Overdue.Filter(("LiabilityType", LiabilityTypes.LIC));
    private IDataCube<ReportVariable> licData => licActuarial + accrual;
    
    private IDataCube<ReportVariable> bop => licData.Filter(("VariableType", AocTypes.BOP), ("Novelty", Novelties.I));
    private IDataCube<ReportVariable> delta => (licData.Filter(("VariableType","!BOP"),("VariableType","!EOP")) + licData.Filter(("VariableType", AocTypes.BOP), ("Novelty", Novelties.N)))
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(x => Math.Abs(x.Value) >= Precision, x => x with { Novelty = Novelties.C, VariableType = "D" });
    private IDataCube<ReportVariable> eop => licData.Filter(("VariableType",AocTypes.EOP));
    
    IDataCube<ReportVariable> Lic => bop + delta + eop;
}

<a id='lrc-actuarial'></a>
## LRC Actuarial (Actuarial Liability for Remaining Coverage)

Actuarial Liability for Remaining Coverage (LRC) report includes all the contributions from [Fulfillment cash flow](#fulfillment-cash-flows), [Contructual Sevice Margin](#technical-margin), [Loss Component](#technical-margin), and [Loss Recovery Component](#technical-margin).

In [0]:
public interface LrcActuarial: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> Fcf => GetScope<CurrentFcf>(Identity).CurrentFcf.Filter(("LiabilityType", LiabilityTypes.LRC));
    private IDataCube<ReportVariable> Csm => GetScope<Csm>(Identity).Csm;
    private IDataCube<ReportVariable> Loreco => GetScope<Loreco>(Identity).Loreco;
    
    IDataCube<ReportVariable> LrcActuarial => Fcf + Csm + Loreco;
}

<a id='lrc'></a>
## LRC (Liability for Remaining Coverage)

Liability for Remaining Coverage (LRC) report includes all the contributions from [LRC Actuarial](#lrc-actuarial) (Fulfillment Cash flow, Contructual Sevice Margin, Loss Component, Loss Recovery Component) and, [Advance](#written-accrual-deferral) Actual, and [Overdue](#written-accrual-deferral) Actual.

In [0]:
public interface Lrc: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> lrcActuarial => GetScope<LrcActuarial>(Identity).LrcActuarial;
    private IDataCube<ReportVariable> accrual => GetScope<WrittenAndAccruals>(Identity).Advance.Filter(("LiabilityType", LiabilityTypes.LRC)) + 
                                                 GetScope<WrittenAndAccruals>(Identity).Overdue.Filter(("LiabilityType", LiabilityTypes.LRC));
    private IDataCube<ReportVariable> lrcData => lrcActuarial + accrual;
    
    private IDataCube<ReportVariable> bop => lrcData.Filter(("VariableType", AocTypes.BOP), ("Novelty", Novelties.I));
    private IDataCube<ReportVariable> delta => (lrcData.Filter(("VariableType","!BOP"),("VariableType","!EOP")) + lrcData.Filter(("VariableType", AocTypes.BOP), ("Novelty", Novelties.N)))
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(x => Math.Abs(x.Value) >= Precision, x => x with { Novelty = Novelties.C, VariableType = "D" });
    private IDataCube<ReportVariable> eop => lrcData.Filter(("VariableType",AocTypes.EOP));
    
    IDataCube<ReportVariable> Lrc => bop + delta + eop;
}

<a name="ifrs-17-financial-performance"></a>
# IFRS 17 Financial Performance

The IFRS 17 Financial Performance aims at disclosing the deltas (i.e. difference between EoP and BoP, or the Change in Estimate) of the IFRS 17 Balance Sheet items (LRC, LIC) and the relevant incurred cash flows (Premiums, Claims, Expenses, etc...) for the given period.

In addition, the IFRS 17 Financial Performance must split the results into at least 4 distinct sections:
 - Insurance Revenue ($IR$)
 - Insurance Service Expense ($ISE$)
 - Insurance Finance Income/Expense ($IFIE$)
 - Other Comprehensive Income ($OCI$)

These can be structured in a hierarchy, such that aggregation and intermediate results can be shown separately.
Also the granularity for some relevant items (e.g. Gross LRC vs Reinsurance LRC) must be split into the different sections, as the notion of Insurance Revenue and Insurance Service Expense must be consistent.

The overall formulas for each of the sections above are as follows:
$$
\begin{array}{rl}
IR =& -\Delta \text{ Gross Non-Financial LRC } - \text{ Incurred Incoming Cash flows } - \text{ Claims Investment Component } + \text{ Amortization of the Insurance Acquisition Cash flows} - \text{ Experience Adjustment On Premium }
\\
ISE =& -\Delta \text{ Reinsurance Non-Financial LRC } - \Delta \text{ Non-Financial LIC } - \Delta \text{ Loss Component } - \text{ Incurred Outgoing Cash flows } - \text{ Amortization of the Insurance Acquisition Cash flows}
\\
IFIE =& -\Delta \text{ Financial LRC } - \Delta \text{ Financial LIC }
\\
OCI =& \Delta \text{ LRC excl. CSM/LC/LoReCo at Valuation Rates } + \Delta \text{ LIC at Valuation Rates } - \Delta \text{ LRC excl. CSM/LC/LoReCo at Current Rates } - \Delta \text{ LIC at Current Rates }
\end{array}
$$

For CSM, LC and LoReCo (and implicit Amortization Factors / Coverage Units), the yield curve used is according to the methodology choice (e.g. BBA uses Locked-In rates). 

The Fulfillment Cash flows components for LIC and LRC are discounted using the appropriate yield curve (Locked-In or Current) according to the choice of the OCI option and methodology. We can distinguish 2 cases for the Building Block approach (BBA):
 - BBA, OCI option ***disabled***: FCF computed using Current rates
 - BBA, OCI option ***enabled***: FCF computed using Locked-In rates

In the case when the OCI option is disabled, the OCI contribution is zero. Otherwise, we have:

$$
\begin{array}{rl}
OCI =& \Delta \text{ LRC excl. CSM/LC/LoReCo at Locked-In Rates } + \Delta \text{ LIC at Locked-In Rates } - \Delta \text{ LRC excl. CSM/LC/LoReCo at Current Rates } - \Delta \text{ LIC at Current Rates }
\end{array}
$$

Usually, some more details on the items above are desired (e.g. separating the LRC into its components FCF, RA, CSM/LC or explicitly showing the CSM Amortization vs. other financial/non-financial adjustments to the CSM estimation). These are specific to each implementation and depend on the data granularity (e.g. how the Amount Types, AoC steps are detailed and structured). This level of detail is omitted from the generic documentation presented here, but are implicit to the way each item above is calculated. Nevertheless, this extra granularity is available when the reports are generated and sliced by the corresponding dimensions. 

## Fulfillment Cash flows (excluding the Loss Component)

Let us first describe the Balance Sheet items, where we need to disclose the Changes in Estimate in the Financial Performance.
As mentioned, often the LRC and LIC contributions are broken down into smaller parts for disclosure and reconciliation purposes.

The Fulfillment Cashflows (which may include onerous contributions in some interpretations when a Loss Component is defined) contributions to the Financial Performance (Insurance Result and OCI) are detailed below.
The main requisite is to ensure the right mapping of its non-financial and financial components into the appropriate sections of the Financial Performance.

Furthermore, we also need to distinguish the Reinsurance contributions to the LRC, as they are considered to be expenses and not revenue.
The OCI contribution is also performed here, as the only contributions to the OCI come from the FCF under the current methodology assumptions.
To all contributions a sign flip is applied. 

In particular: 
- Financial component includes the AoC Type Interest Accreation, Yield Curve Update, and Credit Risk Update,
- Non-Financial component includes all the remaining AoC Types.
- Oci component includes the difference between the movement occured in the period computed applying the discounting with the current yield curve and the discounting with the locked-in yield curve.

The Financial contributions are reported in 'Financial LIC Changes' for a Group of Contract with Liability Type LIC and to 'Financial LRC Changes' for a Group of Contract with Liability Type LRC.
<br>The Non Financial contributions are reported in 'Non Financial LIC Changes' for a Group of Contract with Liability Type LIC. For a Group of Contract with Liability Type LRC, they are reported in 'Non-Financial LRC Changes (Exc. CSM Amortization)' for a Group of Insurance Contract and in 'Non-Financial Reinsurance LRC Changes (Exc. LC/LoReCo)' for a Group of Reinsurance Contract.
<br>The Oci contributions are reported in the 'Other Comprehensive Income' section under 'Financial LIC Changes' or a Group of Contract with Liability Type LIC and to 'Financial LRC Changes' for a Group of Contract with Liability Type LRC.

In [0]:
public interface FcfChangeInEstimate: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> FcfDeltas => GetScope<Fcf>(Identity).Fcf.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                                                   GetScope<Fcf>(Identity).Fcf.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));
    
    private IDataCube<ReportVariable> CurrentFcfDeltas => GetScope<CurrentFcf>(Identity).CurrentFcf.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                                                          GetScope<CurrentFcf>(Identity).CurrentFcf.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

    // Non-Financial Fp
    private string variableTypeNonFinancial => Identity.Id switch {
            { LiabilityType: LiabilityTypes.LRC, IsReinsurance: false } => "IR5",
            { LiabilityType: LiabilityTypes.LRC, IsReinsurance: true } => "ISE10",
            { LiabilityType: LiabilityTypes.LIC } => "ISE12"
            };
    
    private IDataCube<ReportVariable> NonFinancialFcfDeltas => FcfDeltas
        .Filter(("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU"), ("VariableType", "!FX"));   
    
    IDataCube<ReportVariable> FpNonFinancial => -1 * NonFinancialFcfDeltas
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C, VariableType = variableTypeNonFinancial });    
    
    // Financial Fp
    private string variableTypeFpFinancial => Identity.Id switch {
            { LiabilityType: LiabilityTypes.LRC } => "IFIE1",
            { LiabilityType: LiabilityTypes.LIC } => "IFIE2",
            };
    
    // OCI 
    private string variableTypeOciFinancial => Identity.Id switch {
            { LiabilityType: LiabilityTypes.LRC } => "OCI1",
            { LiabilityType: LiabilityTypes.LIC } => "OCI2",
            };
    
    private IDataCube<ReportVariable> FinancialFcfDeltas => FcfDeltas.Filter(("VariableType", AocTypes.IA)) + 
                                                            FcfDeltas.Filter(("VariableType", AocTypes.YCU)) +
                                                            FcfDeltas.Filter(("VariableType", AocTypes.CRU));
    
    IDataCube<ReportVariable> FpFx => -1 * FcfDeltas
        .Filter(("VariableType", AocTypes.FX))
        .AggregateOver(nameof(Novelty))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C, VariableType = "IFIE3"});
    
    IDataCube<ReportVariable> FpFinancial => -1 * FinancialFcfDeltas
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C, VariableType = variableTypeFpFinancial});

    IDataCube<ReportVariable> OciFx => (FcfDeltas - CurrentFcfDeltas)
        .Filter(("VariableType", AocTypes.FX))
        .AggregateOver(nameof(Novelty))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C, VariableType = "OCI3"});

    IDataCube<ReportVariable> OciFinancial => (FcfDeltas - CurrentFcfDeltas)
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(rv => rv with { Novelty = Novelties.C, VariableType = variableTypeOciFinancial});
}

Subsequently, the Contractual Service Margin (CSM), Loss Component (LC) and Loss Recovery Component (LoReCo) are also divided into non-financial and financial components. In addition the corresponding amortization/releases (CSM Amortization, LC Release and LoReCo Release) are also explicitly shown.

## Contractual Service Margin (CSM)

[CSM](#lrc-technical-margin-csm-lc-loreco) contributes to the 'Insurance Revenue' in the case of a Group of Insurance Contract and to the 'Insurance Service Expense' in the case of a Group of Reinsurance Contract and to the 'Insurance Finance Income/Expense'. To all contributions a sign flip is applied. 
In particular, the CSM figures are separated in three cathegories: 
- Amortization (including to AoC Type Amortization),
- Financial contributions (including the AoC Types Interest Accreation, Yield Curve Update, and Credit Risk Update),
- Non Financial contributions (including all the remaining AoC Types).

<br>The Amortization contribution is reported in 'Insurance Revenue' under 'CSM Amortization' or in 'Insurance Service Expense' under 'Reinsurance CSM Amortization' for a Group of Insurance Contract and a Group of Reinsurance Contract, respectively. 
<br>The Financial contributions are reported in 'Financial LRC Changes'.
<br>The Non Financial contributions are reported in 'Non-Financial LRC Changes (Exc. CSM Amortization)' and 'Non-Financial Reinsurance LRC Changes (Exc. LC/LoReCo)' for a Group of Insurance Contract and a Group of Reinsurance Contract, respectively. 

In [0]:
public interface CsmChangeInEstimate: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    
    private (string amortization, string nonFinancial) variableType => Identity.Id switch {
            { IsReinsurance: false} => ("IR3", "IR5"),
            { IsReinsurance: true } => ("ISE7", "ISE10")
            };
    
    private IDataCube<ReportVariable> Csm => GetScope<Csm>(Identity).Csm.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                                             GetScope<Csm>(Identity).Csm.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));
    
    IDataCube<ReportVariable> Amortization => -1 * Csm.Filter(("VariableType", AocTypes.AM)).SelectToDataCube(v => v with { VariableType = variableType.amortization });
    
    IDataCube<ReportVariable> NonFinancialChanges => -1 * Csm
        .Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU"), ("VariableType", "!FX"))
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = variableType.nonFinancial });

    IDataCube<ReportVariable> Fx => -1 * Csm.Filter(("VariableType", AocTypes.FX))
        .AggregateOver(nameof(Novelty))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IFIE3" });

    IDataCube<ReportVariable> FinancialChanges => -1 * (Csm.Filter(("VariableType", AocTypes.IA)) +
                                                        Csm.Filter(("VariableType", AocTypes.YCU)) +
                                                        Csm.Filter(("VariableType", AocTypes.CRU)) )
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IFIE1" });
}

## Loss Component (LC)

LC contributes to the 'Insurance Service Expense' and to the 'Insurance Finance Income/Expense'. To all contributions a sign flip is applied. 
In particular, the LC figures  are separated in three cathegories: 
- Amortization (including to AoC Type Amortization),
- Financial contributions (including the AoC Types Interest Accreation, Yield Curve Update, and Credit Risk Update),
- Non Financial contributions (including all the remaining AoC Types).

<br>The Amortization contribution is reported in 'Loss Component Release'.
<br>The Financial contributions are reported in 'Financial LRC Changes'.
<br>The Non Financial contributions are reported in 'Loss Component / LoReCo Changes (Exc. Releases)'.

In [0]:
public interface LcChangeInEstimate: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    
    private IDataCube<ReportVariable> Lc => GetScope<Lc>(Identity).Lc.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                                            GetScope<Lc>(Identity).Lc.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));
    
    IDataCube<ReportVariable> Amortization => -1 * Lc.Filter(("VariableType", AocTypes.AM)).SelectToDataCube(v => v with { VariableType = "ISE9" });
    
    IDataCube<ReportVariable> NonFinancialChanges => -1 * Lc
        .Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU"), ("VariableType", "!FX"))
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "ISE11" });
    
    IDataCube<ReportVariable> Fx => -1 * Lc.Filter(("VariableType", AocTypes.FX))
        .AggregateOver(nameof(Novelty))
        .SelectToDataCube(v => v with { VariableType = "IFIE3" });

    IDataCube<ReportVariable> FinancialChanges => -1 * (Lc.Filter(("VariableType", AocTypes.IA)) +
                                                        Lc.Filter(("VariableType", AocTypes.YCU)) +
                                                        Lc.Filter(("VariableType", AocTypes.CRU)) )
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IFIE1" });
}

## Loss Recovery Component (LoReCo)

LoReCo contributes to the 'Insurance Service Expense' and to the 'Insurance Finance Income/Expense'. To all contributions a sign flip is applied. 
In particular, the LoReCo figures  are separated in three cathegories: 
- Amortization (including to AoC Type Amortization),
- Financial contributions (including the AoC Types Interest Accreation, Yield Curve Update, and Credit Risk Update),
- Non Financial contributions (including all the remaining AoC Types).

<br>The Amortization contribution is reported in 'LoReCo Release'.
<br>The Financial contributions are reported in 'Financial LRC Changes'.
<br>The Non Financial contributions are reported in 'Loss Component / LoReCo Changes (Exc. Releases)'.

In [0]:
public interface LorecoChangeInEstimate: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    
    private IDataCube<ReportVariable> Loreco => GetScope<Loreco>(Identity).Loreco.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                                                GetScope<Loreco>(Identity).Loreco.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));
    
    IDataCube<ReportVariable> Amortization => -1 * Loreco.Filter(("VariableType", AocTypes.AM)).SelectToDataCube(v => v with { VariableType = "ISE8" });
    
    IDataCube<ReportVariable> NonFinancialChanges => -1 * Loreco
        .Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU"), ("VariableType", "!FX"))
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "ISE11" });
    
    IDataCube<ReportVariable> Fx => -1 * Loreco.Filter(("VariableType", AocTypes.FX))
        .AggregateOver(nameof(Novelty))
        .SelectToDataCube(v => v with { VariableType = "IFIE3" });

    IDataCube<ReportVariable> FinancialChanges => -1 * (Loreco.Filter(("VariableType", AocTypes.IA)) +
                                                        Loreco.Filter(("VariableType", AocTypes.YCU)) +
                                                        Loreco.Filter(("VariableType", AocTypes.CRU)) )
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IFIE1" });
}

## Incurred Actuals

After the main Balance Sheet items, we need to disclose the Incurred Cash flows (i.e. Effective Actuals) for the period in the Financial Performance (which takes into consideration Write-Off on Accruals).
As another requirement, we exclude any investment components explicitly.

These contributions are splitted by Amount Type to the following sections:
- Premiums contributions are reported under 'Insurance Revenue' in 'Premiums' section or in 'Insurance Service Expense' under 'Reinsurance Premiums' for a Group of Insurance Contract and a Group of Reinsurance Contract, respectively,
- Claims Non-Investment Component contributions are reported under 'Insurance Revenue' in 'Claims' section,
- Claims Investment Component contributions are reported under 'Insurance Service Expense' in 'Acquisition Expenses' section and under 'Insurance Revenue' in 'Exc. Investment Components' section with a sign change,
- Expenses contributions are reported under 'Insurance Service Expense' in 'Expenses' section,
- Commissions contributions are reported under 'Insurance Service Expense' in 'Commissions' section.

In [0]:
public interface IncurredActuals: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> WrittenCashflow => GetScope<WrittenAndAccruals>(Identity).Written.Filter(("VariableType", "CF"));
    private IDataCube<ReportVariable> AdvanceWriteOff => GetScope<WrittenAndAccruals>(Identity).Advance.Filter(("VariableType", "WO"));
    private IDataCube<ReportVariable> OverdueWriteOff => GetScope<WrittenAndAccruals>(Identity).Overdue.Filter(("VariableType", "WO"));
    private IDataCube<ReportVariable> EffectiveActuals => WrittenCashflow -1 * (AdvanceWriteOff + OverdueWriteOff);
    
    private string premiumsVariableType => Identity.Id switch {
            { IsReinsurance: false} => "IR1",
            { IsReinsurance: true } => "ISE1"
            }; 
    
    IDataCube<ReportVariable> Premiums => EffectiveActuals
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.PR))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = premiumsVariableType });
    
    IDataCube<ReportVariable> ClaimsNic => EffectiveActuals
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.NIC))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "ISE2" });
    
    private IDataCube<ReportVariable> ClaimsIco => EffectiveActuals
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.ICO)).ToDataCube();
    
    IDataCube<ReportVariable> ClaimsIcoToIr => ClaimsIco.SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IR2" });
    IDataCube<ReportVariable> ClaimsIcoToIse => (-1 * ClaimsIco).SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "ISE5" });
    
    IDataCube<ReportVariable> Expenses => EffectiveActuals
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AE))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "ISE3" });

    IDataCube<ReportVariable> Commissions => EffectiveActuals
        .Where(x => GetStorage().GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AC))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "ISE4" });
}

## Incurred Deferrals and Acquisition Expenses

The next item are the Deferrals, where the Amortization is the only contribution explicitly shown, but these do not have any direct impact on the Financial Performance under the current methodology assumptions. The Amortization contribution is here considered with its sign changed. This contribution is allocated under 'Insurance Service Expense' in 'Acquisition Expenses' section and under 'Insurance Revenue' in 'Acquistion Expenses Amortization' section with a sign change. 

In [0]:
public interface IncurredDeferrals: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> Deferrals => GetScope<Deferrals>(Identity).Filter(("VariableType", "!BOP"),("VariableType", "!EOP"));
        
    private IDataCube<ReportVariable> Amortization => -1 * Deferrals
        .Filter(("VariableType", AocTypes.AM));
    
    IDataCube<ReportVariable> AmortizationToIr => (-1 * Amortization).SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IR4" });
    IDataCube<ReportVariable> AmortizationToIse => Amortization.SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "ISE6" });
}

## Experience Adjustment on Premium (allocation to CSM vs P&L recognition)

The figures disclosed in the Financial Performance as described above consider full Premiums for past, current and future services (for Best Estimate and Actuals).
However, we applied the Premium Allocation factor in the CSM computations to adjust for the experience. To correct this, we need to exclude the Experience Adjustment on Premium part in the Financial Performance to avoid double counting and it is allocated under 'Insurance Revenue' in the 'Exc. Experience Adjustment on Premiums' section.

In [0]:
public interface ExperienceAdjustmentOnPremium: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private IDataCube<ReportVariable> WrittenPremiumToCsm => -1 * GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.APA)).FxData;
    private IDataCube<ReportVariable> BestEstimatePremiumToCsm => -1 * GetScope<FxData>((Identity.Id, Identity.CurrencyType, EstimateTypes.BEPA)).FxData;
        
    IDataCube<ReportVariable> ExperienceAdjustmentOnPremium => (WrittenPremiumToCsm - BestEstimatePremiumToCsm)
        .AggregateOver(nameof(Novelty), nameof(VariableType))
        .SelectToDataCube(v => v with { Novelty = Novelties.C, VariableType = "IR6" });
}

## Financial Performance

As the last step, all the contributions above are calculated and combined into the Financial Performance, which will be used to generate the final report.

In [0]:
public interface FinancialPerformance: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    
    private IDataCube<ReportVariable> FcfChangeInEstimate => GetScope<FcfChangeInEstimate>(Identity);
    private IDataCube<ReportVariable> CsmChangeInEstimate => GetScope<CsmChangeInEstimate>(Identity);
    private IDataCube<ReportVariable> LcChangeInEstimate => GetScope<LcChangeInEstimate>(Identity);
    private IDataCube<ReportVariable> LorecoChangeInEstimate => GetScope<LorecoChangeInEstimate>(Identity);
    private IDataCube<ReportVariable> IncurredActuals => GetScope<IncurredActuals>(Identity);
    private IDataCube<ReportVariable> IncurredDeferrals => GetScope<IncurredDeferrals>(Identity);
    private IDataCube<ReportVariable> ExperienceAdjustmentOnPremium => GetScope<ExperienceAdjustmentOnPremium>(Identity);
    
    IDataCube<ReportVariable> FinancialPerformance => FcfChangeInEstimate + CsmChangeInEstimate + LcChangeInEstimate + LorecoChangeInEstimate + IncurredActuals + IncurredDeferrals + ExperienceAdjustmentOnPremium;
}